# Significance testing

In [1]:
import numpy as np
import pandas as pd
import os
from checkFissFus2 import get_fiss_fus_all , fiss_fus_reassigned_label
from significance_test import sliding_window_cumsum,regularization,apply_function_to_arrays,convert_to_sig_test, apply_function_folder
import matplotlib.pyplot as plt
import pingouin as pg

## import and prepare data

### import data

In [2]:
dir_path_tox_adjusted = "../nellie_output/toxicity/adjusted"
dir_path_mdivi_adjusted = "../nellie_output/mdivi/adjusted"

dir_path_mdivi_self = "../self_event/mdivi/num_area"
dir_path_tox_self = "../self_event/toxicity/num_area"

dir_tox_meter = "D:/Internship/NTU/my_script/mitometer_output/toxin"
dir_mdivi_meter = "D:/Internship/NTU/my_script/mitometer_output/mdivi"

dir_path_tox = "../nellie_output/toxicity/0.25"
dir_path_mdivi = "../nellie_output/mdivi/0.25"

dir_raw_tox = "D:/Internship/NTU/nellie_output/nellie_output/toxins/"
dir_raw_mdivi = "D:/Internship/NTU/nellie_output/nellie_output/mdivi/"

In [3]:

#mitometer
fission_tox_meter, fusion_tox_meter, fiss_frame_meter, fus_frame_meter = get_fiss_fus_all(dir_tox_meter)
fission_mdivi_meter, fusion_mdivi_meter, fiss_frame_meter_tox, fus_frame_meter_tox = get_fiss_fus_all(dir_mdivi_meter, isBound_to_min= False)

#after area threshold adjustment
fission_tox_adjusted, fusion_tox_adjusted, fiss_frame_meter, fus_frame_meter = get_fiss_fus_all(dir_path_tox_adjusted)
fission_mdivi_adjusted, fusion_mdivi_adjusted, fiss_frame_meter_tox, fus_frame_meter_tox = get_fiss_fus_all(dir_path_mdivi_adjusted, isBound_to_min= False)

#before area threshod adjustment
fission_tox, fusion_tox, fiss_frame_meter, fus_frame_meter = get_fiss_fus_all(dir_path_tox)
fission_mdivi, fusion_mdivi, fiss_frame_meter_tox, fus_frame_meter_tox = get_fiss_fus_all(dir_path_mdivi, isBound_to_min= False)

#self event
fission_mdivi_self, fusion_mdivi_self, fiss_self_frame_mdivi, fus_self_frame_mdivi = get_fiss_fus_all(dir_path_mdivi_self, isBound_to_min= False)
fission_tox_self, fusion_tox_self, fiss_self_frame, fus_self_frame = get_fiss_fus_all(dir_path_tox_self)

#fission fusion from Nellie author 
out_tox = apply_function_folder(dir_raw_tox, keywords="features_components", func=fiss_fus_reassigned_label)
out_mdivi = apply_function_folder(dir_raw_mdivi, keywords="features_components", func=fiss_fus_reassigned_label)

### fission/fusion from Nellie's Author

In [4]:
out_tox_copy = out_tox.copy()
x = out_tox.keys()
for key in x:
    condition = key.split("_")[2].split(".")[0]
    out_tox_copy[condition] = out_tox_copy.pop((key))

In [5]:
out_mdivi_copy = out_mdivi.copy()
x = out_mdivi.keys()
for key in x:
    condition = key.split("_")[1] +"_" +  key.split("_")[2].split(".")[0]
    out_mdivi_copy[condition] = out_mdivi_copy.pop((key))

In [6]:
fission_tox_nellie = [out_tox_copy["control"][2],out_tox_copy["FCCP"][2],out_tox_copy["oligomycin"][2],out_tox_copy["Rotenone"][2]]
fusion_tox_nellie = [out_tox_copy["control"][3],out_tox_copy["FCCP"][3],out_tox_copy["oligomycin"][3],out_tox_copy["Rotenone"][3]]
minimum = np.min([len(x) for x in fission_tox_nellie])
fission_tox_nellie = np.array([x[0:minimum] for x in fission_tox_nellie]).flatten()
fusion_tox_nellie = np.array([x[0:minimum] for x in fusion_tox_nellie]).flatten()

In [7]:
fission_mdivi_nellie = [out_mdivi_copy["control_3"][2],out_mdivi_copy["mdivi_3"][2]]
fusion_mdivi_nellie = [out_mdivi_copy["control_3"][3],out_mdivi_copy["mdivi_3"][3]]

fission_mdivi_nellie_10min = [out_mdivi_copy["control_10min"][2],out_mdivi_copy["mdivi_10min"][2]]
fusion_mdivi_nellie_10min = [out_mdivi_copy["control_10min"][3],out_mdivi_copy["mdivi_10min"][3]]

minimum_normal = np.min([len(x) for x in fission_mdivi_nellie])
minimum_10min = np.min([len(x) for x in fission_mdivi_nellie_10min])

fission_mdivi_nellie = np.array([x[0:minimum_normal] for x in fission_mdivi_nellie]).flatten()
fusion_mdivi_nellie = np.array([x[0:minimum_normal] for x in fusion_mdivi_nellie]).flatten()
fission_mdivi_nellie_10min = np.array([x[0:minimum_10min] for x in fission_mdivi_nellie_10min]).flatten()
fusion_mdivi_nellie_10min = np.array([x[0:minimum_10min] for x in fusion_mdivi_nellie_10min]).flatten()


### fission fusion algo output

In [8]:
len(fusion_mdivi_meter[0]), len(fusion_mdivi_meter[1]), len(fusion_mdivi_meter[2]),  len(fusion_mdivi_meter[3])
fusion_mdivi_meter[2] = fusion_mdivi_meter[2][0:61]
fission_mdivi_meter[2] = fission_mdivi_meter[2][0:61]

In [9]:

# control 10min, control 3, mdivi 10min, mdivi 3
#extract result from mdivi (fission, self_fission, fusion, self_fusion) in 10 second and 1 second interval

# 10 min fission
fission_mdivi_10min = np.array([fission_mdivi[0],fission_mdivi[2] ])
fusion_mdivi_10min = np.array([fusion_mdivi[0],fusion_mdivi[2] ])
fission_mdivi_self_10min = np.array([fission_mdivi_self[0],fission_mdivi_self[2] ])
fusion_mdivi_self_10min = np.array([fusion_mdivi_self[0],fusion_mdivi_self[2] ])
fission_mdivi_10min_meter = np.array([fission_mdivi_meter[0],fission_mdivi_meter[2] ])
fusion_mdivi_10min_meter = np.array([fusion_mdivi_meter[0],fusion_mdivi_meter[2] ])

fission_mdivi_10min_adjusted = np.array([fission_mdivi_adjusted[0],fission_mdivi_adjusted[2] ])
fusion_mdivi_10min_adjusted = np.array([fusion_mdivi_adjusted[0],fusion_mdivi_adjusted[2] ])

# 1.3 second fission
fission_mdivi = np.array([x[0:minimum_normal] for x in [fission_mdivi[1],fission_mdivi[3] ]])
fusion_mdivi = np.array([x[0:minimum_normal] for x in[fusion_mdivi[1],fusion_mdivi[3] ]])
fission_mdivi_self = np.array([x[0:minimum_normal] for x in[fission_mdivi_self[1],fission_mdivi_self[3] ]])
fusion_mdivi_self = np.array([x[0:minimum_normal] for x in[fusion_mdivi_self[1],fusion_mdivi_self[3] ]])
fission_mdivi_meter = np.array([x[0:minimum_normal] for x in[fission_mdivi_meter[1],fission_mdivi_meter[3] ]])
fusion_mdivi_meter = np.array([x[0:minimum_normal] for x in[fusion_mdivi_meter[1],fusion_mdivi_meter[3] ]])

fission_mdivi_adjusted = np.array([x[0:minimum_normal] for x in[fission_mdivi_adjusted[1],fission_mdivi_adjusted[3] ]])
fusion_mdivi_adjusted =np.array([x[0:minimum_normal] for x in [fusion_mdivi_adjusted[1],fusion_mdivi_adjusted[3] ]])



#all event
fission_mdivi_all = fission_mdivi + fission_mdivi_self
fusion_mdivi_all = fusion_mdivi + fusion_mdivi_self

fission_mdivi_10min_all = fission_mdivi_10min + fission_mdivi_self_10min
fusion_mdivi_10min_all = fusion_mdivi_10min + fusion_mdivi_self_10min

minimum_tox = min(len(fission_tox[0]), len(fission_tox[1]), len(fission_tox[2]), len(fission_tox[3]))

### Area and label number

In [10]:
all_label = []
all_area = []
all_index = []
all_raw_label = []
label_path = "../check_label&area/toxicity/"

files= os.listdir(label_path)
all_name = [files[i][0:5] for i in range(len(files))]

for file in files:
    df = pd.read_csv(label_path + file)
    index = np.expand_dims(np.arange(len(df)),axis=1)
    
    df_label = np.array(df[['label_num']])
    #df_label = np.hstack((index, df_label))

    df_area = np.array(df[['area']])
    #df_area = np.hstack((index, df_area))

    df_raw_label = np.array(df[['raw_label']])

    all_label.append(df_label)
    all_area.append(df_area)
    all_index.append(index)
    all_raw_label.append(df_raw_label)

## fission fusion ratio before and after the area threshold adjustment for toxicity data

In [429]:
# 
print(np.sum(fission_tox_adjusted[0]),np.sum(fission_tox_adjusted[1]), np.sum(fission_tox_adjusted[2]), np.sum(fission_tox_adjusted[3]))
print(np.sum(fusion_tox_adjusted[0]),np.sum(fusion_tox_adjusted[1]), np.sum(fusion_tox_adjusted[2]), np.sum(fusion_tox_adjusted[3]))

print(np.sum(fission_tox_adjusted[0]) / np.sum(fusion_tox_adjusted[0]) ,np.sum(fission_tox_adjusted[1]) / np.sum(fusion_tox_adjusted[1]), 
      np.sum(fission_tox_adjusted[2]) / np.sum(fusion_tox_adjusted[3]), np.sum(fission_tox_adjusted[3]) / np.sum(fusion_tox_adjusted[3]))

759 1245 619 503
616 914 479 441
1.2321428571428572 1.3621444201312911 1.4036281179138321 1.1405895691609977


In [430]:
print(np.sum(fission_tox[0]),np.sum(fission_tox[1]), np.sum(fission_tox[2]), np.sum(fission_tox[3]))
print(np.sum(fusion_tox[0]),np.sum(fusion_tox[1]), np.sum(fusion_tox[2]), np.sum(fusion_tox[3]))

print(np.sum(fission_tox[0]) / np.sum(fusion_tox[0]) ,np.sum(fission_tox[1]) / np.sum(fusion_tox[1]), 
      np.sum(fission_tox[2]) / np.sum(fusion_tox[3]), np.sum(fission_tox[3]) / np.sum(fusion_tox[3]))


880 1300 712 549
883 1327 714 603
0.9966024915062288 0.9796533534287868 1.1807628524046434 0.9104477611940298


## fission fusion ratio before and after area threshold adjustment for MDIVI

In [ ]:
#order: control , mdivi , control 10min, mdivi 10 min
print("fission", np.sum(fission_mdivi_adjusted[0]),np.sum(fission_mdivi_adjusted[1]), np.sum(fission_mdivi_10min_adjusted[0]), np.sum(fission_mdivi_10min_adjusted[1]))
print("fusion",np.sum(fusion_mdivi_adjusted[0]),np.sum(fusion_mdivi_adjusted[1]), np.sum(fission_mdivi_10min_adjusted[0]), np.sum(fission_mdivi_10min_adjusted[1]))

print("control , mdivi, control 10min, mdivi 10min")
print(np.sum(fission_mdivi_adjusted[0]) / np.sum(fusion_mdivi_adjusted[0]) ,np.sum(fission_mdivi_adjusted[1]) / np.sum(fusion_mdivi_adjusted[1]), 
      np.sum(fission_mdivi_10min_adjusted[0]) / np.sum(fusion_mdivi_10min_adjusted[0]), np.sum(fission_mdivi_10min_adjusted[1]) / np.sum(fission_mdivi_10min_adjusted[1]))

fission 394 367 216 217
fusion 322 286 104 190
control , mdivi, control 10min, mdivi 10min
1.2236024844720497 1.2832167832167831 2.076923076923077 1.1421052631578947


In [34]:
#order: control , mdivi , control 10min, mdivi 10 min
print("fission",np.sum(fission_mdivi[0]),np.sum(fission_mdivi[1]), np.sum(fission_mdivi_10min[0]), np.sum(fission_mdivi_10min[1]))
print("fusion",np.sum(fusion_mdivi[0]),np.sum(fusion_mdivi[1]), np.sum(fusion_mdivi_10min[0]), np.sum(fusion_mdivi_10min[1]))

print("control , mdivi, control 10min, mdivi 10min")
print(np.sum(fission_mdivi[0]) / np.sum(fusion_mdivi[0]) ,np.sum(fission_mdivi[1]) / np.sum(fusion_mdivi[1]), 
      np.sum(fission_mdivi_10min[0]) / np.sum(fusion_mdivi_10min[0]), np.sum(fission_mdivi_10min[1]) / np.sum(fusion_mdivi_10min[1]))

fission 300 457 300 262
fusion 286 470 286 298
control , mdivi, control 10min, mdivi 10min
1.048951048951049 0.9723404255319149 1.048951048951049 0.8791946308724832


## Regularization and data preparation

In [554]:
#column name
toxins = np.repeat(['control', 'FCCP', 'oligomycin', 'rotenone'], minimum_tox)
toxins_mdivi = np.repeat(['control', 'mdivi'], minimum_normal)
toxins_mdivi_10min = np.repeat(['control_10min', 'mdivi_10min'], minimum_10min)
assess_method_tox = np.repeat(['algorithm' , 'mitometer'], minimum_tox*4)
assess_mdivi = np.repeat(['algorithm' , 'mitometer'], minimum_normal*4)
assess_mdivi_10min= np.repeat(['algorithm' , 'mitometer'], minimum_10min*4)

### toxins

In [555]:
# data preparation with regularization (0 -> epsilon)
# window == 62 137

apply_regularization = False
epsilon = 0.01

fusion_tox,fusion_tox_pd = convert_to_sig_test(fusion_tox , toxins, regularization_=False, epsilon_=epsilon)
fission_tox, fission_tox_pd = convert_to_sig_test(fission_tox, toxins, regularization_=False, epsilon_=epsilon)

fusion_tox_adjusted,fusion_tox_adjusted_pd = convert_to_sig_test(fusion_tox_adjusted , toxins, regularization_=False, epsilon_=epsilon)
fission_tox_adjusted, fission_tox_adjusted_pd = convert_to_sig_test(fission_tox_adjusted, toxins, regularization_=False, epsilon_=epsilon)

fission_tox_self , fission_tox_self_pd= convert_to_sig_test(fission_tox_self,toxins, regularization_=False, epsilon_=epsilon)
fusion_tox_self, fusion_tox_self_pd = convert_to_sig_test(fusion_tox_self, toxins,regularization_=False, epsilon_=epsilon)

fusion_tox_meter , fusion_tox_meter_pd = convert_to_sig_test(fusion_tox_meter, toxins,regularization_=False, epsilon_=epsilon)
fission_tox_meter, fission_tox_meter_pd = convert_to_sig_test(fission_tox_meter,toxins, regularization_=False, epsilon_=epsilon)

fission_tox_nellie = pd.DataFrame({"toxin": toxins,'event': fission_tox_nellie})
fusion_tox_nellie = pd.DataFrame({"toxin": toxins,'event': fusion_tox_nellie})

fission_tox_all = fission_tox + fission_tox_self
fusion_tox_all = fusion_tox + fusion_tox_self   
fission_tox_all_pd = pd.DataFrame({"toxin": toxins,'event': fission_tox_all})
fusion_tox_all_pd = pd.DataFrame({"toxin": toxins,'event': fusion_tox_all})

fission_tox_adjusted_all = fission_tox_adjusted + fission_tox_self
fusion_tox_adjusted_all = fusion_tox_adjusted + fusion_tox_self   
fission_tox_adjusted_all_pd = pd.DataFrame({"toxin": toxins,'event': fission_tox_adjusted_all})
fusion_tox_adjusted_all_pd = pd.DataFrame({"toxin": toxins,'event': fusion_tox_adjusted_all})

#FF ratio
ff_ratio = fission_tox / np.where(fusion_tox == 0, 1, fusion_tox)
ff_ratio_all = fission_tox_all/np.where(fusion_tox_all == 0, 1, fusion_tox_all)
ff_ratio_self = fission_tox_self/np.where(fusion_tox_self == 0, 1, fusion_tox_self)
ff_ratio_meter_tox = fission_tox_meter/ fusion_tox_meter
ff_ratio_adjusted = fission_tox_adjusted / np.where(fusion_tox_adjusted == 0, 1, fusion_tox_adjusted)

ff_ratio_pd = pd.DataFrame({"toxin": toxins,'event': ff_ratio})
ff_ratio_all_pd = pd.DataFrame({"toxin": toxins,'event': ff_ratio_all})
ff_ratio_self_pd = pd.DataFrame({"toxin": toxins,'event': ff_ratio_self})
ff_ratio_meter_pd = pd.DataFrame({"toxin": toxins,'event': ff_ratio_meter_tox})
ff_ratio_adjusted_pd = pd.DataFrame({"toxin": toxins,'event': ff_ratio_adjusted})

C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\2653447523.py:36: RuntimeWarning: divide by zero encountered in divide
  ff_ratio_meter_tox = fission_tox_meter/ fusion_tox_meter
C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\2653447523.py:36: RuntimeWarning: invalid value encountered in divide
  ff_ratio_meter_tox = fission_tox_meter/ fusion_tox_meter


### MDIVI

In [556]:

apply_regularization = False
epsilon = 0.01

''' 60s data'''
fusion_mdivi , fusion_mdivi_pd= convert_to_sig_test(fusion_mdivi, col_value=toxins_mdivi,regularization_=False,epsilon_=epsilon)
fission_mdivi, fission_mdivi_pd = convert_to_sig_test(fission_mdivi, col_value= toxins_mdivi, regularization_=False,epsilon_=epsilon)
#new adjust data
fusion_mdivi_adjusted , fusion_mdivi_adjusted_pd= convert_to_sig_test(fusion_mdivi_adjusted, col_value=toxins_mdivi,regularization_=False,epsilon_=epsilon)
fission_mdivi_adjusted, fission_mdivi_adjusted_pd = convert_to_sig_test(fission_mdivi_adjusted, col_value= toxins_mdivi, regularization_=False,epsilon_=epsilon)

fusion_mdivi_self , fusion_mdivi_self_pd= convert_to_sig_test(fusion_mdivi_self, col_value=toxins_mdivi,regularization_=False,epsilon_=epsilon)
fission_mdivi_self, fission_mdivi_self_pd = convert_to_sig_test(fission_mdivi_self, col_value=toxins_mdivi,regularization_=False,epsilon_=epsilon)

fusion_mdivi_meter, fusion_mdivi_meter_pd = convert_to_sig_test(fusion_mdivi_meter,col_value=toxins_mdivi, regularization_=False,epsilon_=epsilon)
fission_mdivi_meter,fission_mdivi_meter_pd  = convert_to_sig_test(fission_mdivi_meter, col_value=toxins_mdivi, regularization_=False,epsilon_=epsilon)

'''10 min data'''
fission_mdivi_10min , fission_mdivi_10min_pd= convert_to_sig_test(fission_mdivi_10min, col_value= toxins_mdivi_10min,regularization_=False,epsilon_=epsilon)
fusion_mdivi_10min , fusion_mdivi_10min_pd= convert_to_sig_test(fusion_mdivi_10min,col_value= toxins_mdivi_10min, regularization_=False,epsilon_=epsilon)

fission_mdivi_10min_adjusted , fission_mdivi_10min_adjusted_pd= convert_to_sig_test(fission_mdivi_10min_adjusted, col_value= toxins_mdivi_10min,regularization_=False,epsilon_=epsilon)
fusion_mdivi_10min_adjusted , fusion_mdivi_10min_adjusted_pd= convert_to_sig_test(fusion_mdivi_10min_adjusted,col_value= toxins_mdivi_10min, regularization_=False,epsilon_=epsilon)

fission_mdivi_self_10min , fission_mdivi_self_10min_pd= convert_to_sig_test(fission_mdivi_self_10min,col_value=toxins_mdivi_10min, regularization_=False,epsilon_=epsilon)
fusion_mdivi_self_10min, fusion_mdivi_self_10min_pd = convert_to_sig_test(fusion_mdivi_self_10min,col_value=toxins_mdivi_10min, regularization_=False,epsilon_=epsilon)

fission_mdivi_10min_meter ,fission_mdivi_meter_10min_pd = convert_to_sig_test(fission_mdivi_10min_meter, col_value=toxins_mdivi_10min,regularization_=False,epsilon_=epsilon)
fusion_mdivi_10min_meter, fusion_mdivi_meter_10min_pd = convert_to_sig_test(fusion_mdivi_10min_meter,col_value=toxins_mdivi_10min, regularization_=False,epsilon_=epsilon)


In [557]:
print(fission_mdivi.shape)
print(fission_mdivi_self.shape)

print(toxins_mdivi_10min.shape)
print(fission_mdivi_nellie_10min.shape)

(278,)
(278,)
(122,)
(122,)


In [558]:
fission_mdivi_all_10min = fission_mdivi_10min + fission_mdivi_self_10min
fusion_mdivi_all_10min = fusion_mdivi_10min + fusion_mdivi_self_10min
fission_mdivi_all = fission_mdivi + fission_mdivi_self
fusion_mdivi_all = fusion_mdivi + fusion_mdivi_self

fission_mdivi_all_pd = pd.DataFrame({"toxin": toxins_mdivi,'event': fission_mdivi_all})
fusion_mdivi_all_pd = pd.DataFrame({"toxin": toxins_mdivi,'event': fusion_mdivi_all})
fission_mdivi_all_10min_pd = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': fission_mdivi_all_10min})
fusion_mdivi_all_10min_pd = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': fusion_mdivi_all_10min})

fission_mdivi_nellie = pd.DataFrame({"toxin": toxins_mdivi,'event': fission_mdivi_nellie})
fusion_mdivi_nellie = pd.DataFrame({"toxin": toxins_mdivi,'event': fusion_mdivi_nellie})
fission_mdivi_nellie_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': fission_mdivi_nellie_10min})
fusion_mdivi_nellie_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': fusion_mdivi_nellie_10min})

#for ff ratio
ff_ratio_mdivi = fission_mdivi / np.where(fusion_mdivi == 0, 1, fusion_mdivi)
ff_ratio_mdivi_adjusted = fission_mdivi_adjusted / np.where(fusion_mdivi_adjusted == 0, 1, fusion_mdivi_adjusted)
ff_ratio_all_mdivi = fission_mdivi_all/np.where(fusion_mdivi_all == 0, 1, fusion_mdivi_all)
ff_ratio_self_mdivi = fission_mdivi_self/np.where(fusion_mdivi_self == 0, 1, fusion_mdivi_self)
ff_ratio_mito_mdivi = fission_mdivi_meter / fusion_mdivi_meter 

ff_ratio_mdivi_10min = fission_mdivi_10min / np.where(fusion_mdivi_10min == 0, 1, fusion_mdivi_10min) 
ff_ratio_mdivi_adjusted_10min = fission_mdivi_10min_adjusted /np.where(fusion_mdivi_10min_adjusted == 0, 1, fusion_mdivi_10min_adjusted)  
ff_ratio_all_mdivi_10min = fission_mdivi_all_10min/np.where(fusion_mdivi_all_10min == 0, 1, fusion_mdivi_all_10min) 
ff_ratio_self_mdivi_10min = fission_mdivi_self_10min/np.where(fusion_mdivi_self_10min == 0, 1, fusion_mdivi_self_10min) 
ff_ratio_mito_mdivi_10min = fission_mdivi_10min_meter / fusion_mdivi_10min_meter

ff_ratio_mdivi = pd.DataFrame({"toxin": toxins_mdivi,'event': ff_ratio_mdivi})
ff_ratio_mdivi_adjusted = pd.DataFrame({"toxin": toxins_mdivi,'event': ff_ratio_mdivi_adjusted})
ff_ratio_all_mdivi = pd.DataFrame({"toxin": toxins_mdivi,'event': ff_ratio_all_mdivi})
ff_ratio_self_mdivi = pd.DataFrame({"toxin": toxins_mdivi,'event': ff_ratio_self_mdivi})
ff_ratio_mito_mdivi = pd.DataFrame({"toxin": toxins_mdivi,'event': ff_ratio_mito_mdivi})

ff_ratio_mdivi_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': ff_ratio_mdivi_10min})
ff_ratio_mdivi_adjusted_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': ff_ratio_mdivi_adjusted_10min})
ff_ratio_all_mdivi_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': ff_ratio_all_mdivi_10min})
ff_ratio_self_mdivi_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': ff_ratio_self_mdivi_10min})
ff_ratio_mito_mdivi_10min = pd.DataFrame({"toxin": toxins_mdivi_10min,'event': ff_ratio_mito_mdivi_10min})

C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\1807279383.py:21: RuntimeWarning: divide by zero encountered in divide
  ff_ratio_mito_mdivi = fission_mdivi_meter / fusion_mdivi_meter
C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\1807279383.py:21: RuntimeWarning: invalid value encountered in divide
  ff_ratio_mito_mdivi = fission_mdivi_meter / fusion_mdivi_meter
C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\1807279383.py:27: RuntimeWarning: divide by zero encountered in divide
  ff_ratio_mito_mdivi_10min = fission_mdivi_10min_meter / fusion_mdivi_10min_meter
C:\Users\iceji\AppData\Local\Temp\ipykernel_15784\1807279383.py:27: RuntimeWarning: invalid value encountered in divide
  ff_ratio_mito_mdivi_10min = fission_mdivi_10min_meter / fusion_mdivi_10min_meter


## Statistical Analysis
Assumption before using ANOVA = 
1. Normality: each group should be normally distributed
2. Homogeneity of variance: the variance of the residuals should be approximately equal for all groups
3. Independent observations: the observations in each group must be independent of each other

### Statistical analysis of toxins with my algorithm


#### 1. check assumption

In [473]:
# fission
normality = pg.normality(data = fission_tox_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fission_tox_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                   W          pval  normal
toxin                                     
control     0.968105  2.685394e-03   False
FCCP        0.917609  4.214017e-07   False
oligomycin  0.811430  5.477192e-12   False
rotenone    0.926134  1.443198e-06   False
               W          pval  equal_var
levene  16.05322  5.172778e-10      False


In [474]:
#fusion
normality = pg.normality(data = fusion_tox_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_tox_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                   W          pval  normal
toxin                                     
control     0.976081  1.639644e-02   False
FCCP        0.814869  7.298806e-12   False
oligomycin  0.797365  1.754231e-12   False
rotenone    0.955406  1.999841e-04   False
                W          pval  equal_var
levene  17.404154  8.383049e-11      False


In [475]:
# self ff ratio
normality = pg.normality(data = ff_ratio_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = ff_ratio_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                   W          pval  normal
toxin                                     
control          NaN  1.000000e+00    True
FCCP        0.834626  4.567181e-11   False
oligomycin       NaN  1.000000e+00    True
rotenone         NaN  1.000000e+00    True
         W  pval  equal_var
levene NaN   NaN      False


In [476]:
#mitometer
normality = pg.normality(data = fusion_tox_meter_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_tox_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                   W          pval  normal
toxin                                     
control     0.418419  5.069909e-21   False
FCCP        0.337935  3.074460e-22   False
oligomycin  0.406667  3.307738e-21   False
rotenone    0.586101  5.303815e-18   False
                W          pval  equal_var
levene  17.404154  8.383049e-11      False


In [ ]:
#from author
normality = pg.normality(data = fusion_tox_nellie, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_tox_nellie, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                   W          pval  normal
toxin                                     
control     0.721830  8.564392e-15   False
FCCP        0.632281  5.195695e-17   False
oligomycin  0.734259  1.903872e-14   False
rotenone    0.636430  6.443203e-17   False
               W      pval  equal_var
levene  1.450961  0.227057       True


#### 2. Since the data are neither normal or having a equal variance, we will use kruskal

Significance when P < 0.05

In [478]:
#before area threshold 
# Chi square test score with DOF 3 and confidence 0.005 = 12.838
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_all_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_all_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_self_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_self_pd))
print(" ")
print(" adjusted val")
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_adjusted_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_adjusted_pd))

print(" ")
print(" mitometer")
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_meter_pd))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_meter_pd))

print(" ")
print(" fission_fusion from author") # no significance diff from each group
print(pg.kruskal(dv = 'event', between =  'toxin', data = fission_tox_nellie))
print(pg.kruskal(dv = 'event', between =  'toxin', data = fusion_tox_nellie))
# all P is less than 0.05  = all significant for four group of data (rank signed test are all diff)
# reject null hypothesis, => means are different for each class (control, FCCP, oligomycin, rotenone)

        Source  ddof1           H         p-unc
Kruskal  toxin      3  141.446254  1.843361e-30
        Source  ddof1           H         p-unc
Kruskal  toxin      3  309.264552  9.830112e-67
        Source  ddof1           H         p-unc
Kruskal  toxin      3  118.797858  1.400645e-25
        Source  ddof1           H         p-unc
Kruskal  toxin      3  299.814119  1.091437e-64
        Source  ddof1           H         p-unc
Kruskal  toxin      3  293.827554  2.155829e-63
        Source  ddof1           H         p-unc
Kruskal  toxin      3  280.861015  1.378889e-60
 
 adjusted val
        Source  ddof1           H         p-unc
Kruskal  toxin      3  149.885528  2.789074e-32
        Source  ddof1          H         p-unc
Kruskal  toxin      3  69.520212  5.407429e-15
 
 mitometer
        Source  ddof1          H         p-unc
Kruskal  toxin      3  56.227515  3.756577e-12
        Source  ddof1         H     p-unc
Kruskal  toxin      3  19.84347  0.000183
 
 fission_fusion from auth

#### 3. Pairwise test for 

    3.1 fission & fusion before and after adjustment
    3.2 fission & fusion all before and after adjustment
    3.3 fission & fusion from mitometer  
    3.4 fission & fusion from Author script? 

##### fission & fusion, adjusted fission & fusion, 

In [480]:
print("fission") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_pd, parametric=False))

fission
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  12972.5   two-sided   
1    toxin        FCCP  oligomycin   False       False  14902.5   two-sided   
2    toxin        FCCP    rotenone   False       False  16044.5   two-sided   
3    toxin     control  oligomycin   False       False  11919.0   two-sided   
4    toxin     control    rotenone   False       False  13918.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  12018.5   two-sided   

          p-unc    hedges  
0  4.005135e-08  0.719592  
1  2.913402e-17  1.010290  
2  2.173094e-24  1.342023  
3  9.910068e-05  0.410057  
4  3.532692e-12  0.877543  
5  5.096362e-05  0.435099  


In [481]:
print("adjusted fission") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_adjusted_pd, parametric=False))

adjusted fission
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  13847.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  15440.0   two-sided   
2    toxin        FCCP    rotenone   False       False  16133.0   two-sided   
3    toxin     control  oligomycin   False       False  11968.0   two-sided   
4    toxin     control    rotenone   False       False  13338.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  11522.0   two-sided   

          p-unc    hedges  
0  8.278303e-12  0.883908  
1  1.737931e-20  1.143507  
2  5.256822e-25  1.372996  
3  7.127939e-05  0.390731  
4  1.295858e-09  0.736712  
5  9.884513e-04  0.337511  


In [482]:
print("fusion") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_pd, parametric=False))

fusion
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  12709.5   two-sided   
1    toxin        FCCP  oligomycin   False       False  14731.5   two-sided   
2    toxin        FCCP    rotenone   False       False  15588.5   two-sided   
3    toxin     control  oligomycin   False       False  12048.0   two-sided   
4    toxin     control    rotenone   False       False  13153.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  10785.0   two-sided   

          p-unc    hedges  
0  3.595313e-07  0.680055  
1  2.625353e-16  0.927720  
2  2.176230e-21  1.152408  
3  4.379838e-05  0.388110  
4  7.499338e-09  0.728548  
5  3.135341e-02  0.279210  


In [483]:
print("adjusted fusion") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_adjusted_pd, parametric=False))

adjusted fusion
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  11703.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  13579.5   two-sided   
2    toxin        FCCP    rotenone   False       False  13968.0   two-sided   
3    toxin     control  oligomycin   False       False  11853.5   two-sided   
4    toxin     control    rotenone   False       False  12310.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  10052.5   two-sided   

          p-unc    hedges  
0  3.799473e-04  0.524266  
1  1.232068e-10  0.771274  
2  2.088923e-12  0.844828  
3  1.457693e-04  0.404589  
4  6.887215e-06  0.527587  
5  3.033447e-01  0.117255  


##### Fission all & fusion all (with self fission / fusion)

In [484]:
print("fission all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_all_pd, parametric=False))

fission all
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  17638.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  18018.0   two-sided   
2    toxin        FCCP    rotenone   False       False  18283.5   two-sided   
3    toxin     control  oligomycin   False       False  12030.0   two-sided   
4    toxin     control    rotenone   False       False  14999.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  13347.0   two-sided   

          p-unc    hedges  
0  2.347827e-36  1.975137  
1  1.264395e-39  2.313399  
2  5.355037e-42  2.731295  
3  5.256215e-05  0.463075  
4  9.485018e-18  1.134044  
5  1.347043e-09  0.719766  


In [485]:
print("fission adjusted all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_adjusted_all_pd, parametric=False))

fission adjusted all
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  17839.5   two-sided   
1    toxin        FCCP  oligomycin   False       False  18108.0   two-sided   
2    toxin        FCCP    rotenone   False       False  18277.5   two-sided   
3    toxin     control  oligomycin   False       False  11982.5   two-sided   
4    toxin     control    rotenone   False       False  14805.0   two-sided   
5    toxin  oligomycin    rotenone   False       False  13056.5   two-sided   

          p-unc    hedges  
0  4.535529e-38  2.094018  
1  2.001761e-40  2.404891  
2  6.011068e-42  2.762994  
3  7.118206e-05  0.450082  
4  1.178115e-16  1.067153  
5  1.931177e-08  0.666577  


In [486]:
print("fusion all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_all_pd, parametric=False))

fusion all
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  17770.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  18054.0   two-sided   
2    toxin        FCCP    rotenone   False       False  18278.5   two-sided   
3    toxin     control  oligomycin   False       False  11648.5   two-sided   
4    toxin     control    rotenone   False       False  14308.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  12736.0   two-sided   

          p-unc    hedges  
0  1.719140e-37  1.994281  
1  5.864976e-40  2.247352  
2  5.921704e-42  2.554216  
3  5.305908e-04  0.395257  
4  5.201186e-14  0.924753  
5  2.966336e-07  0.552846  


In [487]:
print("fusion adjusted all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_adjusted_all_pd, parametric=False))

fusion adjusted all
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  17638.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  17970.5   two-sided   
2    toxin        FCCP    rotenone   False       False  18076.0   two-sided   
3    toxin     control  oligomycin   False       False  11528.0   two-sided   
4    toxin     control    rotenone   False       False  13919.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  12387.5   two-sided   

          p-unc    hedges  
0  2.222980e-36  1.937536  
1  3.079332e-39  2.176325  
2  3.792825e-40  2.381554  
3  1.028182e-03  0.389669  
4  4.042380e-12  0.810678  
5  4.332942e-06  0.469190  


##### Fission fusion from mitometer

In [491]:
print("fission mitometer") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_meter_pd, parametric=False))

fission mitometer
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  13272.5   two-sided   
1    toxin        FCCP  oligomycin   False       False  12536.0   two-sided   
2    toxin        FCCP    rotenone   False       False  10952.5   two-sided   
3    toxin     control  oligomycin   False       False   8606.0   two-sided   
4    toxin     control    rotenone   False       False   6989.0   two-sided   
5    toxin  oligomycin    rotenone   False       False   7755.0   two-sided   

          p-unc    hedges  
0  1.545791e-11  0.827871  
1  8.304499e-08  0.642336  
2  9.553068e-03  0.344610  
3  1.205285e-01 -0.207947  
4  1.190099e-05 -0.549661  
5  3.848848e-03 -0.335201  


In [489]:
print("fusion mitometer") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_meter_pd, parametric=False))

fusion mitometer
  Contrast           A           B  Paired  Parametric   U-val alternative  \
0    toxin        FCCP     control   False       False  8957.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  9031.5   two-sided   
2    toxin        FCCP    rotenone   False       False  7665.5   two-sided   
3    toxin     control  oligomycin   False       False  9460.5   two-sided   
4    toxin     control    rotenone   False       False  8082.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  8005.0   two-sided   

      p-unc    hedges  
0  0.242317 -0.167300  
1  0.327657 -0.135699  
2  0.000096 -0.515472  
3  0.846050  0.033647  
4  0.004733 -0.380774  
5  0.002577 -0.410264  


##### fission fusion from nellie

In [559]:
print("fission nellie") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fission_tox_nellie, parametric=False))

fission nellie
  Contrast           A           B  Paired  Parametric   U-val alternative  \
0    toxin        FCCP     control   False       False  9737.5   two-sided   
1    toxin        FCCP  oligomycin   False       False  9591.0   two-sided   
2    toxin        FCCP    rotenone   False       False  9534.0   two-sided   
3    toxin     control  oligomycin   False       False  9295.0   two-sided   
4    toxin     control    rotenone   False       False  9193.0   two-sided   
5    toxin  oligomycin    rotenone   False       False  9257.0   two-sided   

      p-unc    hedges  
0  0.562350  0.241834  
1  0.733129  0.117618  
2  0.808087  0.216145  
3  0.883703 -0.130909  
4  0.756942 -0.026099  
5  0.835985  0.104001  


In [493]:
print("fusion nellie") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = fusion_tox_nellie, parametric=False))

fusion nellie
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False   9686.5   two-sided   
1    toxin        FCCP  oligomycin   False       False   9375.0   two-sided   
2    toxin        FCCP    rotenone   False       False   9993.5   two-sided   
3    toxin     control  oligomycin   False       False   9046.0   two-sided   
4    toxin     control    rotenone   False       False   9738.0   two-sided   
5    toxin  oligomycin    rotenone   False       False  10061.5   two-sided   

      p-unc    hedges  
0  0.620818  0.210180  
1  0.988238  0.103245  
2  0.312035  0.182241  
3  0.580427 -0.127642  
4  0.558297 -0.021260  
5  0.262824  0.097177  


##### fission / fusion ratio

In [ ]:
print("fission fusion ratio") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_pd, parametric=False))

fusion nellie
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False   9540.5   two-sided   
1    toxin        FCCP  oligomycin   False       False   9351.5   two-sided   
2    toxin        FCCP    rotenone   False       False  10036.0   two-sided   
3    toxin     control  oligomycin   False       False   9235.5   two-sided   
4    toxin     control    rotenone   False       False   9930.0   two-sided   
5    toxin  oligomycin    rotenone   False       False  10062.5   two-sided   

      p-unc    hedges  
0  0.812515 -0.046525  
1  0.960449 -0.034104  
2  0.320659 -0.023997  
3  0.820642  0.016766  
4  0.405573  0.023766  
5  0.300845  0.008470  


In [562]:
print("fission fusion ratio adjusted") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_adjusted_pd, parametric=False))

fission fusion ratio adjusted
  Contrast           A           B  Paired  Parametric    U-val alternative  \
0    toxin        FCCP     control   False       False  10187.0   two-sided   
1    toxin        FCCP  oligomycin   False       False   9848.0   two-sided   
2    toxin        FCCP    rotenone   False       False  10568.5   two-sided   
3    toxin     control  oligomycin   False       False   9079.5   two-sided   
4    toxin     control    rotenone   False       False   9789.0   two-sided   
5    toxin  oligomycin    rotenone   False       False  10083.0   two-sided   

      p-unc    hedges  
0  0.221186  0.255388  
1  0.479950  0.192338  
2  0.070996  0.261485  
3  0.642121 -0.068589  
4  0.537543  0.006250  
5  0.286639  0.075015  


In [563]:
print("fission fusion ratio all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_all_pd, parametric=False))

fission fusion ratio all
  Contrast           A           B  Paired  Parametric   U-val alternative  \
0    toxin        FCCP     control   False       False  9356.0   two-sided   
1    toxin        FCCP  oligomycin   False       False  9721.0   two-sided   
2    toxin        FCCP    rotenone   False       False  9803.0   two-sided   
3    toxin     control  oligomycin   False       False  9694.0   two-sided   
4    toxin     control    rotenone   False       False  9771.5   two-sided   
5    toxin  oligomycin    rotenone   False       False  9402.5   two-sided   

      p-unc    hedges  
0  0.965943  0.094950  
1  0.608361  0.136510  
2  0.523799  0.064856  
3  0.637469  0.054495  
4  0.555551 -0.024607  
5  0.978703 -0.069950  


### Statistical testing of MDIVI with my algorithm

#### 1. Check assumption

In [494]:
# fission mdivi
normality = pg.normality(data = fission_mdivi_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fission_mdivi_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                W          pval  normal
toxin                                  
control  0.894909  1.831708e-08   False
mdivi    0.872055  1.335167e-09   False
               W      pval  equal_var
levene  4.491263  0.034961      False


In [496]:
normality = pg.normality(data = fusion_mdivi_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_mdivi_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                W          pval  normal
toxin                                  
control  0.932055  3.068881e-06   False
mdivi    0.867810  8.487915e-10   False
               W      pval  equal_var
levene  2.288583  0.131473       True


In [495]:
normality = pg.normality(data = fission_mdivi_10min_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fission_mdivi_10min_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                      W      pval  normal
toxin                                    
control_10min  0.971320  0.161929    True
mdivi_10min    0.948128  0.011754   False
               W     pval  equal_var
levene  0.974486  0.32555       True


In [497]:
normality = pg.normality(data = fusion_mdivi_10min_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_mdivi_10min_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                      W      pval  normal
toxin                                    
control_10min  0.974518  0.232228    True
mdivi_10min    0.925335  0.001141   False
               W      pval  equal_var
levene  2.091866  0.150692       True


In [498]:
normality = pg.normality(data = fusion_mdivi_meter_pd, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = fusion_mdivi_meter_pd, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)

                W          pval  normal
toxin                                  
control  0.559831  1.159752e-18   False
mdivi    0.404555  2.195639e-21   False
               W      pval  equal_var
levene  6.242931  0.013051      False


#### 2. Statistical testing with Mann-Whitney

In [511]:
x = int((fusion_mdivi.shape[0])/2)
# control = x, mdivi = y
print("10s interval ")
print(pg.mwu(fusion_mdivi[0:x] , fusion_mdivi[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi[0:x] , fission_mdivi[x:-1], alternative='two-sided'))

print(pg.mwu(fusion_mdivi_adjusted[0:x] , fusion_mdivi_adjusted[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_adjusted[0:x] , fission_mdivi_adjusted[x:-1], alternative='two-sided'))

print(" ")
print(" 1 min interval")
x = int((fusion_mdivi_10min.shape[0])/2)
print(pg.mwu(fusion_mdivi_10min[0:x] , fusion_mdivi_10min[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_10min[0:x] , fission_mdivi_10min[x:-1], alternative='two-sided'))

print(pg.mwu(fusion_mdivi_10min_adjusted[0:x] , fusion_mdivi_10min_adjusted[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_10min_adjusted[0:x] , fission_mdivi_10min_adjusted[x:-1], alternative='two-sided'))

10s interval 
       U-val alternative     p-val     RBC     CLES
MWU  11209.0   two-sided  0.013898  0.1687  0.58435
       U-val alternative     p-val       RBC      CLES
MWU  10806.0   two-sided  0.064841  0.126681  0.563341
       U-val alternative     p-val       RBC      CLES
MWU  10922.5   two-sided  0.042347  0.138828  0.569414
       U-val alternative     p-val       RBC      CLES
MWU  10635.0   two-sided  0.111955  0.108852  0.554426
 
 1 min interval
      U-val alternative     p-val       RBC      CLES
MWU  1815.5   two-sided  0.941486 -0.007923  0.496038
      U-val alternative     p-val       RBC      CLES
MWU  2184.0   two-sided  0.064136  0.193443  0.596721
      U-val alternative     p-val      RBC      CLES
MWU  1237.5   two-sided  0.001836 -0.32377  0.338115
      U-val alternative     p-val       RBC      CLES
MWU  1919.5   two-sided  0.639562  0.048907  0.524454


In [513]:
print("10s interval ")
print(pg.mwu(fusion_mdivi_all[0:x] , fusion_mdivi_all[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_all[0:x] , fission_mdivi_all[x:-1], alternative='two-sided'))


print(" ")
print(" 1 min interval")
print(pg.mwu(fusion_mdivi_all_10min[0:x] , fusion_mdivi_all_10min[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_all_10min[0:x] , fission_mdivi_all_10min[x:-1], alternative='two-sided'))

10s interval 
      U-val alternative     p-val       RBC      CLES
MWU  7134.0   two-sided  0.321286  0.082878  0.541439
      U-val alternative     p-val       RBC      CLES
MWU  6316.5   two-sided  0.622321 -0.041211  0.479394
 
 1 min interval
     U-val alternative     p-val       RBC      CLES
MWU  977.5   two-sided  0.000009 -0.465847  0.267077
     U-val alternative     p-val       RBC      CLES
MWU  950.0   two-sided  0.000005 -0.480874  0.259563


In [ ]:
x = int((fusion_mdivi.shape[0])/2)
# control = x, mdivi = y
print(pg.mwu(fusion_mdivi_meter[0:x] , fusion_mdivi_meter[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_meter[0:x] , fission_mdivi_meter[x:-1], alternative='two-sided'))

print(pg.mwu(fusion_mdivi_all[0:x] , fusion_mdivi_all[x:-1], alternative='two-sided'))
print(pg.mwu(fission_mdivi_all[0:x] , fission_mdivi_all[x:-1], alternative='two-sided'))


       U-val alternative     p-val       RBC     CLES
MWU  10698.0   two-sided  0.014368  0.115421  0.55771
       U-val alternative     p-val       RBC      CLES
MWU  10522.5   two-sided  0.063727  0.097122  0.548561


#### Fission fusion ratio


In [566]:
print("fission fusion ratio") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_mdivi, parametric=False))

print(" ")
print("fission fusion ratio 10 min") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_mdivi_10min, parametric=False))

fission fusion ratio
  Contrast        A      B  Paired  Parametric   U-val alternative     p-unc  \
0    toxin  control  mdivi   False       False  9753.0   two-sided  0.890563   

     hedges  
0 -0.227179  
 
fission fusion ratio 10 min
  Contrast              A            B  Paired  Parametric   U-val  \
0    toxin  control_10min  mdivi_10min   False       False  2073.0   

  alternative     p-unc    hedges  
0   two-sided  0.277183 -0.072235  


In [567]:
print("fission fusion ratio adjusted") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_mdivi_adjusted, parametric=False))

print(" ")
print("fission fusion ratio 10 min adjusted") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_mdivi_adjusted_10min, parametric=False))

fission fusion ratio adjusted
  Contrast        A      B  Paired  Parametric    U-val alternative     p-unc  \
0    toxin  control  mdivi   False       False  10080.0   two-sided  0.530682   

     hedges  
0 -0.112976  
 
fission fusion ratio 10 min adjusted
  Contrast              A            B  Paired  Parametric   U-val  \
0    toxin  control_10min  mdivi_10min   False       False  2285.0   

  alternative     p-unc   hedges  
0   two-sided  0.029476  0.02839  


In [568]:
print("fission fusion ratio all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_all_mdivi, parametric=False))

print(" ")
print("fission fusion ratio 10 min all") # shows all of then are significantly different 
#check the influence and correlation with Rank Biserial Correlation
print(pg.pairwise_tests(dv = 'event', between =  'toxin', data = ff_ratio_all_mdivi_10min, parametric=False))

fission fusion ratio all
  Contrast        A      B  Paired  Parametric   U-val alternative     p-unc  \
0    toxin  control  mdivi   False       False  9365.0   two-sided  0.659733   

     hedges  
0 -0.012396  
 
fission fusion ratio 10 min all
  Contrast              A            B  Paired  Parametric   U-val  \
0    toxin  control_10min  mdivi_10min   False       False  1927.5   

  alternative     p-unc    hedges  
0   two-sided  0.733345 -0.015599  


## Try two way anava analyze significance difference between two algorithm

In [56]:
#np.mean(ff_ratio_mito_mdivi[0:x])
#np.mean(ff_ratio_mito_mdivi[x:-1])

df_ff_all = pd.concat((df_ff_ratio ,df_ff_ratio_mito))


normality = pg.normality(data = df_ff_all, dv = 'event', group = 'toxin')
homogeneity = pg.homoscedasticity(data = df_ff_all, dv = 'event', group = 'toxin')
print(normality)
print(homogeneity)


                   W          pval  normal
toxin                                     
control     0.176764  6.816448e-33   False
FCCP        0.578623  2.772587e-25   False
oligomycin  0.354072  5.145356e-30   False
rotenone    0.436756  1.885349e-28   False
              W      pval  equal_var
levene  5.00955  0.001875      False


In [66]:
df_ff_all["assessment method"] = assess_method_tox
df_ff_all

,toxin,event,assessment method
0,control,1.000000,algorithm
1,control,0.200000,algorithm
2,control,0.900000,algorithm
3,control,3.500000,algorithm
4,control,0.800000,algorithm
...,...,...,...
543,rotenone,0.005000,mitometer
544,rotenone,0.333333,mitometer
545,rotenone,0.003333,mitometer
546,rotenone,0.005000,mitometer


In [71]:
pg.anova(data= df_ff_all, dv = "event", between= ['toxin' , 'assessment method'])

,Source,SS,DF,MS,F,p-unc,np2
0,toxin,1.118693e+05,3,37289.774238,5.411805,1.070070e-03,0.014703
1,assessment method,4.165545e+05,1,416554.458726,60.453875,1.739071e-14,0.052639
2,toxin * assessment method,2.446709e+05,3,81556.954993,11.836229,1.243662e-07,0.031605
3,Residual,7.496811e+06,1088,6890.450889,NaN,NaN,NaN


In [74]:
pg.pairwise_tukey(data= df_ff_all, dv = "event", between= 'assessment method')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,algorithm,mitometer,10.289456,49.280119,-38.990662,5.11851,-7.617581,0.0,-0.459879


In [75]:
pg.pairwise_tukey(data= df_ff_all, dv = "event", between= 'toxin')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,FCCP,control,45.061731,21.788031,23.273700,7.384502,3.151695,0.009035,0.236466
1,FCCP,oligomycin,45.061731,19.646674,25.415057,7.384502,3.441675,0.003357,0.343601
2,FCCP,rotenone,45.061731,32.642714,12.419016,7.384502,1.681768,0.333804,0.146130
3,control,oligomycin,21.788031,19.646674,2.141357,7.384502,0.289980,0.991508,0.024309
4,control,rotenone,21.788031,32.642714,-10.854684,7.384502,-1.469928,0.456126,-0.111299
5,oligomycin,rotenone,19.646674,32.642714,-12.996040,7.384502,-1.759908,0.293417,-0.178590
